In [ ]:
from helpers import *
from gaussfit import *

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

## Configuration

In [ ]:
# OLD LIST OF PMTS THAT WERE OFF FOR RUN-1 AND RUN-2
# list of PMTs at 0 voltage from HV files + disconnected pmts: 290
offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
offChs = PMTid_to_channel(offPMTs)
print("OLD OFFs: {}".format(offChs))

# NEW LIST OF PMTS THAT ARE OFF FOR RUN-3
offPMTs2 = [143, 254, 290 ]
offChs2 = PMTid_to_channel(offPMTs2)
print("CURRENT OFFs: {}".format(offChs2))

## PMTs that USED TO BE OFF AND HAVE BEEN SUCCESSFULLY RECOVERED
recoPMTs = [1, 166, 192, 230, 238, 222, 302 ]
recoChs = PMTid_to_channel(recoPMTs)

## PMTs that are ON but CANNOT BE EQUALIZED (>2000 V)
nochangePMTs = [ 111, 309, 340, 353 ]
nochangeChs = PMTid_to_channel(nochangePMTs)
print("NOT EQUALIZED: {}".format(nochangeChs))

## last drifting channels 
newPMTsWest = [ 66, 71 ]
newDriftChs = PMTid_to_channel(newPMTsWest)

WESTonly = [ x for x in range(1,181) ]
chWESTonly = PMTid_to_channel(WESTonly)

EASTonly = [ x for x in range(181,361) ]
chEASTonly = PMTid_to_channel(EASTonly)

In [ ]:
srcpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"
histpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/quick/"
figpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/figs/"

# equalization
#file = srcpath + "equalization_run10890_1697390445.csv" #nominal
#file = srcpath + "equalization_run10893_1697472021.csv" #+30V
#file = srcpath + "equalization_run10894_1697477246.csv" #-30V
#file = srcpath + "equalization_run10895_1697481155.csv" #-50V
#file = srcpath + "equalization_run10896_1697485836.csv" #-100V

#file = srcpath + "backgroundphotons_run11749_1709385772.csv"
#file = srcpath + "backgroundphotons_run11590_1708018817.csv"
#file = srcpath + "backgroundphotons_run11809_1710626100.csv"
#file = srcpath + "backgroundphotons_run11836_1711955795.csv"

file = getMostRecentCalibration(srcpath)

## PMT gain distribution

In [ ]:
print( "Loading data from: "+ file )
run=getRun(file)
timestamp=getTimestamp(file)
data = getDataFrame(file, offPMTs2, False)

fitOutput  = fitGainsDistribution( data, BINSIZE=0.002,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.01) )

plt.figure(dpi=150)
plt.xlim((0.2, 0.6))
plt = makeplot(timestamp, figpath+"gain_hist_"+str(run)+".pdf", *fitOutput )
plt.title("PMT Gain Equalization", fontsize=14)
name = "run"+str(run)+"_histo_all"
#plt.savefig("figs/"+name+".png", dpi=100)
#plt.savefig(name+".pdf", dpi=100)
plt.show()

## Scatter plots

In [ ]:
fig = plt.figure(dpi=100)

plt.scatter(data["pmt"],data["q"],marker="o")

## latest equalization target
plt.axhline(y=0.40, color="red", linestyle='dotted')

## KNOWN HV ISSUES
_selDrift = data['pmt'].isin(newDriftChs)
#plt.scatter(data[_selDrift]["pmt"],data[_selDrift]["q"],marker="x",color="C1", label="HV issues (West slot3)")
## RECOVERED FULLY
_selRecov = data['pmt'].isin(recoChs)
#plt.scatter(data[_selRecov]['pmt'],data[_selRecov]['q'],marker="x",color="magenta", label="Fully recovered")
# ON BUT NOT EQUALIZED
_selNoeq = data['pmt'].isin(nochangeChs)
plt.scatter(data[_selNoeq]['pmt'],data[_selNoeq]['q'],marker="x",color="red", label="Not equalized")

plt.axvline(x=90,color="magenta",linestyle="dotted")
plt.axvline(x=180,color="magenta",linestyle="dotted")
plt.axvline(x=270,color="magenta",linestyle="dotted")

#plt.xlim((180,360))
#plt.ylim((0.2,0.6))
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.legend()
plt.grid()
plt.show()

#plt.savefig("figs/run"+str(run)+"_scatter_all.png", dpi=200)

In [ ]:
fig = plt.figure(dpi=150)

plt.scatter(data["id"],data["q"],marker="o")

## latest equalization target
plt.axhline(y=0.40, color="red", linestyle='dotted')

## KNOWN HV ISSUES
_selDrift = data['pmt'].isin(newDriftChs)
#plt.scatter(data[_selDrift]["id"],data[_selDrift]["q"],marker="x",color="C1", label="HV issues (West slot3)")
## RECOVERED FULLY
_selRecov = data['pmt'].isin(recoChs)
#plt.scatter(data[_selRecov]['id'],data[_selRecov]['q'],marker="x",color="magenta", label="Fully recovered")
# ON BUT NOT EQUALIZED
_selNoeq = data['pmt'].isin(nochangeChs)
plt.scatter(data[_selNoeq]['id'],data[_selNoeq]['q'],marker="x",color="red", label="Not equalized")

plt.axvline(x=90,color="magenta",linestyle="dotted")
plt.axvline(x=180,color="magenta",linestyle="dotted")
plt.axvline(x=270,color="magenta",linestyle="dotted")

#plt.xlim((180,360))
#plt.ylim((0.35,0.45))
plt.xlabel("PMT ID")
plt.ylabel("Gains [10^7 electrons]")
plt.legend()
plt.grid()
plt.show()

In [ ]:
## PRINTING OUTLIERS
reference = 0.39
low_bound = reference - 0.01
high_bound = reference + 0.01

_selHigh = data.q>high_bound
_selLow = data.q<low_bound


#print("PMT channels above "+ str(high_bound))
#print( data[_selHigh][["pmt","id","q"]] )

print("PMT channels below "+ str(low_bound))
print( data[_selLow][["pmt","id","q"]] )

## Comparisons and random stuff

In [ ]:
file0 = srcpath + "backgroundphotons_run11505_1706910209.csv" #nominal
file1 = srcpath + "backgroundphotons_run11566_1707776993.csv" #-50V
file2 = srcpath + "backgroundphotons_run11567_1707781418.csv" #-30V
file3 = srcpath + "backgroundphotons_run11568_1707782812.csv" #+30V
file4 = srcpath + "backgroundphotons_run11571_1707792247.csv" #+50V

remove = offPMTs2 + nochangePMTs

#data_01 = getDataFrame(file01, remove, False)
data_0 = getDataFrame(file0, remove, False)
data_1 = getDataFrame(file1, remove, False)
data_2 = getDataFrame(file2, remove, False)
data_3 = getDataFrame(file3, remove, False)
data_4 = getDataFrame(file4, remove, False)

plt.figure(dpi=200)

#man_data_01 = data_01[data_01['pmt'].isin(manualChs)]
#man_data_0 = data_0[data_0['pmt'].isin(manualChs)]
#plt.scatter(man_data_01['pmt'],man_data_01['q'],marker="s", label="Start (10263)")
#plt.scatter(man_data_0['pmt'],man_data_0['q'],marker="s", label="Start (10313)")

#plt.scatter(data_01['pmt'],data_01['q'],marker="s", label="Start (10263)")
plt.scatter(data_0['id'],data_0['q'],marker="s", label="Nominal (11505)")
plt.scatter(data_1['id'],data_1['q'],marker="s", label="-50V (11566)")
plt.scatter(data_2['id'],data_2['q'],marker="s", label="-30V (11567)")
plt.scatter(data_3['id'],data_3['q'],marker="s", label="+30V (11568-11569)")
plt.scatter(data_4['id'],data_4['q'],marker="s", label="+50V (11571)")

plt.axhline(y=0.4, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
#plt.xlim((0, 180))
#plt.savefig("equalization_runs.png",dpi=100)

In [ ]:
plt.figure(dpi=200)

up=0.6
low=0.25
BINSIZE=0.002
nbins = int((up-low)/BINSIZE)

plt.hist(data_0['q'],range=(low,up), bins=nbins, histtype="step", lw=2, label="Nominal (11505)")
plt.hist(data_1['q'],range=(low,up), bins=nbins, histtype="step", lw=2, label="-50V (11566)")
plt.hist(data_2['q'],range=(low,up), bins=nbins, histtype="step", lw=2, label="-30V (11567)")
plt.hist(data_3['q'],range=(low,up), bins=nbins, histtype="step", lw=2, label="+30V (11568-11569)")
plt.hist(data_4['q'],range=(low,up), bins=nbins, histtype="step", lw=2, label="+50V (11571)")

plt.axvline(x=0.4, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("Gain [10^7 electrons]")
plt.grid()
#plt.xlim((0, 180))
#plt.savefig("equalization_runs.png",dpi=100)

In [ ]:
file01 = srcpath + "equalization_run10263_1691610710.csv" #nominal 
file0 = srcpath + "equalization_run10313_1692203859.csv" #nominal 

remove = offPMTs + EASTonly

data_01 = getDataFrame(file01, remove, False)
data_0 = getDataFrame(file0, remove, False)

#man_data_01 = data_01[data_01['pmt'].isin(manualChs)]
#man_data_0 = data_0[data_0['pmt'].isin(manualChs)]
#plt.scatter(man_data_01['pmt'],man_data_01['q'],marker="s", label="Start (10263)")
#plt.scatter(man_data_0['pmt'],man_data_0['q'],marker="s", label="Start (10313)")

plt.scatter(data_01['pmt'],data_01['q'],marker="s", label="Start (10263)")
plt.scatter(data_0['pmt'],data_0['q'],marker="s", label="Start (10313)")

plt.axhline(y=0.45, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((180.0, 359))
plt.savefig("manual_fix_highG.png",dpi=100)

In [ ]:
file1 = srcpath +"backgroundphotons_run10040_1688176609.csv"
file2 = srcpath +"backgroundphotons_run10058_1688566398.csv"
file3 = srcpath +"newicaruscode_run10058_1688574998.csv"
file4 = srcpath +"backgroundphotons_run10058_1688566368.csv"

data1 = getDataFrame(file1, offPMTs, False)
data2 = getDataFrame(file2, offPMTs, False)
data3 = getDataFrame(file3, offPMTs, False)
data4 = getDataFrame(file4, offPMTs, False)

In [ ]:
#plt.scatter(data1['pmt'],data1['q'],marker="o", label="10040 - new ")
plt.scatter(data2['pmt'],data2['q'],marker="o", label="10040 - old")
plt.scatter(data3['pmt'],data3['q'],marker="o", alpha=0.5, label="10058")
plt.scatter(data4['pmt'],data3['q'],marker="o", alpha=0.5, label="10097")

plt.axhline(y=0.45, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((0.0, 359))
plt.ylim((0.3, 0.5))

In [ ]:
#fitOutput1  = fitGainsDistribution( data1, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput2  = fitGainsDistribution( data2, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput3  = fitGainsDistribution( data3, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput4  = fitGainsDistribution( data4, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )

fig = plt.figure(figsize=(10, 5))
plt.xlim((0.3, 0.6))
#plt = makeplot(timestamp, figpath+"gain_1hist_"+str(run)+".pdf", *fitOutput1 )
plt = makeplot(timestamp, figpath+"gain_2hist_"+str(run)+".pdf", *fitOutput2 )
plt = makeplot(timestamp, figpath+"gain_3hist_"+str(run)+".pdf", *fitOutput3 )
plt = makeplot(timestamp, figpath+"gain_4hist_"+str(run)+".pdf", *fitOutput4 )

plt.savefig("runs_diff_files.png", dpi=900)

### Scan for recovered PMTs

In [ ]:
files = [ "backgroundphotons_run11246_1702593081.csv", "backgroundphotons_run11258_1702673047.csv",
          "backgroundphotons_run11260_1702678720.csv", "backgroundphotons_run11262_1702685886.csv",
          "backgroundphotons_run11265_1702694286.csv" ] 

In [ ]:
fig = plt.figure(figsize=(8,5),dpi=100)

for f in files:
    
    filename = srcpath + f
    runnum = filename.split("_")[1]
    df = pd.read_csv(filename)

    _sel = df.pmt.isin(recoChs)

    plt.scatter(df[_sel].pmt,df[_sel].q,marker="o",label=runnum)

plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.legend()
plt.grid()

## Compare old-new code

In [ ]:
fileold = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/backgroundphotons_run11590_1708018817.csv"
#filenew1 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/test/csvs/backgroundphotons_run11590_1708021616.csv"
filenew2 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/test/backgroundphotons_run11590_1708021616.csv"

fileold2 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/old_calibrationdb/backgroundphotons_run9342_1672496767.csv"
filenew3 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/test/backgroundphotons_run9342_1672516694.csv"
#filenew5 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/test/T0.5/tstart0.1_f2_n2/backgroundphotons_run11590_1708022816.csv"
#filenew6 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/test/noT/tstart0.1_f2_n2/backgroundphotons_run11590_1708022817.csv"

In [ ]:
dataold = getDataFrame(fileold, offPMTs2, False)
#datanew1 = getDataFrame(filenew1, offPMTs2, False)
datanew2 = getDataFrame(filenew2, offPMTs2, False)

dataold2 = getDataFrame(fileold2, offPMTs, False)
datanew3 = getDataFrame(filenew3, offPMTs, False)
#datanew6 = getDataFrame(filenew6, offPMTs2, False)

In [ ]:
fig = plt.figure(dpi=200)
#plt.scatter(dataold['pmt'],dataold['q'],marker="o", alpha=0.5, label="11590 - Old fitting")
#plt.scatter(datanew1['pmt'],datanew1['q'],marker="o", alpha=0.5, label="11590 - NEW")
#plt.scatter(datanew2['pmt'],datanew2['q'],marker="o", alpha=0.7, label="11590 - New fitting")
plt.scatter(dataold2['pmt'],dataold2['q'],marker="o", label="9342 - Old fitting")
plt.scatter(datanew3['pmt'],datanew3['q'],marker="o", alpha=0.5, label="9342 - New fitting")
#plt.scatter(datanew5['pmt'],datanew5['q'],marker="o", alpha=0.5, label="11590 - T0.5, r=0.1, n=2")
#plt.scatter(datanew6['pmt'],datanew6['q'],marker="o", alpha=0.5, label="11590 - noT, 0.1, n=2")

plt.axvline(x=90,color="black",linestyle="dashed")
plt.axvline(x=180,color="black",linestyle="dashed")
plt.axvline(x=270,color="black",linestyle="dashed")

plt.axhline(y=0.40, color="red", linestyle='')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((0.0, 359))
#plt.ylim((0.35, 0.45))
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

#fitOutputold  = fitGainsDistribution( dataold, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
#timestampold=getTimestamp(fileold)
#fitOutputnew  = fitGainsDistribution( datanew2, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
#timestampnew=getTimestamp(filenew2)
fitOutputold  = fitGainsDistribution( dataold2, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
timestampold=getTimestamp(fileold2)
fitOutputnew  = fitGainsDistribution( datanew3, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
timestampnew=getTimestamp(filenew3)


plt = makeplot(timestampold, figpath+"test.pdf", *fitOutputold )
plt = makeplot(timestampnew, figpath+"test.pdf", *fitOutputnew )

plt.xlim((0.2, 0.6))

plt.show()